In [2]:
%load_ext autoreload
%autoreload 2

from MyTSP import TSP
import numpy as np

In [4]:
txt = """
- 3 7 5
3 - 2 3
5 5 - 7
4 2 4 -
"""

names = ["c1", "c2", "c3", "c4"]

tsp = TSP()
tsp.from_text(names, txt)

print(tsp.get_matrix())

[[nan  3.  7.  5.]
 [ 3. nan  2.  3.]
 [ 5.  5. nan  7.]
 [ 4.  2.  4. nan]]


### compute the cost of a tour

In [9]:
tour = ["c1", "c2", "c3", "c4", "c1"]
for i, name in enumerate(tour):
    tour[i] = tsp.name_to_index(name)

cost = tsp.compute_cost(tour)
print(f"cost={cost}")

cost=16.0


### nearest neighbour

In [38]:
start = tsp.name_to_index("c1")
tour = tsp.search_nearest(start=start, complete=True)
tsp.print_tour(tour)

tour (index) -> [0 1 2 3 0]
tour (names) -> ['c1', 'c2', 'c3', 'c4', 'c1']
index(0 -> 1) | name(c1 -> c2) | cost is 3.0
index(1 -> 2) | name(c2 -> c3) | cost is 2.0
index(2 -> 3) | name(c3 -> c4) | cost is 7.0
index(3 -> 0) | name(c4 -> c1) | cost is 4.0
total cost is 16.0


### pilot search

In [44]:
start = tsp.name_to_index("c1")
tour = tsp.search_pilot(start=start, complete=True)
tsp.print_tour(tour)

current tour: i=[0], n=['c1'], cost=0
looking for best point out of [1, 2, 3]
s=1, i=[1, 0, 3, 2, 0], n=['c2', 'c1', 'c4', 'c3', 'c1'], c=17.0
s=2, i=[2, 0, 1, 3, 0], n=['c3', 'c1', 'c2', 'c4', 'c1'], c=15.0
s=3, i=[3, 0, 1, 2, 0], n=['c4', 'c1', 'c2', 'c3', 'c1'], c=14.0
best is s=3
current tour: i=[0, 3], n=['c1', 'c4'], cost=5.0
looking for best point out of [1, 2]
s=1, i=[1, 0, 3, 2, 0], n=['c2', 'c1', 'c4', 'c3', 'c1'], c=17.0
s=2, i=[2, 0, 3, 1, 0], n=['c3', 'c1', 'c4', 'c2', 'c1'], c=15.0
best is s=2
current tour: i=[0, 3, 2], n=['c1', 'c4', 'c3'], cost=9.0
looking for best point out of [1]
s=1, i=[1, 0, 2, 3, 0], n=['c2', 'c1', 'c3', 'c4', 'c1'], c=21.0
best is s=1
tour (index) -> [0 3 2 1 0]
tour (names) -> ['c1', 'c4', 'c3', 'c2', 'c1']
index(0 -> 3) | name(c1 -> c4) | cost is 5.0
index(3 -> 2) | name(c4 -> c3) | cost is 4.0
index(2 -> 1) | name(c3 -> c2) | cost is 5.0
index(1 -> 0) | name(c2 -> c1) | cost is 3.0
total cost is 17.0


### exhaustive search

In [52]:
start = tsp.name_to_index("c1")
tour = tsp.exhaustive_search(start=start, complete=True, k=9999)
tsp.print_tour(tour)

searching through 6 tours
tour (index) -> [0 3 1 2 0]
tour (names) -> ['c1', 'c4', 'c2', 'c3', 'c1']
index(0 -> 3) | name(c1 -> c4) | cost is 5.0
index(3 -> 1) | name(c4 -> c2) | cost is 2.0
index(1 -> 2) | name(c2 -> c3) | cost is 2.0
index(2 -> 0) | name(c3 -> c1) | cost is 5.0
total cost is 14.0


In [ ]:
# https://en.wikipedia.org/wiki/3-opt

def all_segments(n: int):
    """Generate all segments combinations"""
    return ((i, j, k)
        for i in range(n)
        for j in range(i + 2, n)
        for k in range(j + 2, n + (i > 0)))

def three_opt_segment(tour, i, j, k, allow_reverse):
    """If reversing tour[i:j] would make the tour shorter, then do it."""
    # Given tour [...A-B...C-D...E-F...]
    A, B, C, D, E, F = tour[i-1], tour[i], tour[j-1], tour[j], tour[k-1], tour[k % len(tour)]
    d0 = distance(mat, A, B) + distance(mat, C, D) + distance(mat, E, F)
    d1 = distance(mat, A, C) + distance(mat, B, D) + distance(mat, E, F)
    d2 = distance(mat, A, B) + distance(mat, C, E) + distance(mat, D, F)
    d3 = distance(mat, A, D) + distance(mat, E, B) + distance(mat, C, F)
    d4 = distance(mat, F, B) + distance(mat, C, D) + distance(mat, E, A)

    if allow_reverse and d0 > d1:
        tour[i:j] = reversed(tour[i:j])
        return -d0 + d1
    elif allow_reverse and d0 > d2:
        tour[j:k] = reversed(tour[j:k])
        return -d0 + d2
    elif allow_reverse and d0 > d4:
        tour[i:k] = reversed(tour[i:k])
        return -d0 + d4
    elif d0 > d3:
        tmp = tour[j:k] + tour[i:j]
        tour[i:k] = tmp
        return -d0 + d3
    return 0

def three_opt(tour, allow_reverse):
    """Iterative improvement based on 3 exchange."""
    while True:
        delta = 0
        for (a, b, c) in all_segments(len(tour)):
            delta += three_opt_segment(tour, a, b, c, allow_reverse)
        if delta >= 0:
            break
    return tour




In [ ]:
# non-reversing three-opt

def three_opt_non_reversing(tour, i, j, k):
    tour = tour.copy()
    tmp = np.concatenate((tour[j+1:k+1], tour[i+1:j+1]))
    tour[i+1:k+1] = tmp
    return tour

tour = [1,2,3,4,5,6,7,8,1]
tour = np.array(tour)
print(tour)

tour = exam_to_python(tour)
i = exam_to_python(2) # first index of 1st edge 
j = exam_to_python(4) # first index of 2nd edge
k = exam_to_python(6) # first index of 3rd edge
tour = three_opt_non_reversing(tour, i, j, k)
tour = python_to_exam(tour)

print(tour)